# Study colors in ALMA

In [1]:
import sys
sys.path.append("../../")

import pandas as pd
from src.colors import color_alma as color

path="../../data/interim/ALMA/"

## Collect colors by instance

In [2]:
FILES=!ls $path | grep -v count

In [3]:
acc=[]

for f in FILES:
    temp_dict={}
    instance=pd.read_csv(path+f)
    instance['color'] = instance['logtext'].apply(color)
    for i, r in instance.iterrows():
        if r['color'] not in temp_dict.keys():
            temp_dict[r['color']] = 1
        else:
            temp_dict[r['color']] += 1
    for k in temp_dict.keys():
        acc.append( {'file': f, 'color': k, 'count': temp_dict[k]} )

colors_by_instance = pd.DataFrame(acc)

## Colors statistics

In [20]:
colors_by_instance

,file,color,count,count-norm
0,2017-07-01T19:07:07.456-da62-AntObs.csv,Request load AntInterferometryController,1,0.001587
1,2017-07-01T19:07:07.456-da62-AntObs.csv,Full path alma ACS DEC2016 ACSSW lib libAntInt...,1,0.001587
2,2017-07-01T19:07:07.456-da62-AntObs.csv,Loaded alma ACS DEC2016 ACSSW lib libAntInterf...,1,0.001587
3,2017-07-01T19:07:07.456-da62-AntObs.csv,Switched state component CONTROL ANTxx AntInte...,1,0.001587
4,2017-07-01T19:07:07.456-da62-AntObs.csv,Switched state component CONTROL ANTxx AntInte...,1,0.001587
...,...,...,...,...
31224,2017-07-12T16:09:50.075-da43-AntObs.csv,Operation Timeout type {} code {},2,0.003175
31225,2017-07-12T16:09:50.075-da43-AntObs.csv,Problem manipulating hardware type {} code {} ...,2,0.003175
31226,2017-07-12T16:09:50.075-da43-AntObs.csv,More events came in from NC than receiver proc...,1,0.001587
31227,2017-07-12T16:09:50.075-da43-AntObs.csv,Switched state component CONTROL ANTxx AntInte...,1,0.001587


In [4]:
# of instances:
len(FILES)

630

In [5]:
# of colors
len(colors_by_instance["color"].unique())

171

In [6]:
# Normalize by # of instaces
colors_by_instance['count-norm'] = colors_by_instance['count'] / len(FILES)

In [7]:
# Normalized totals by color (AVERAGE!)
norm_total = colors_by_instance[["color", "count-norm"]].groupby(["color"]).sum().sort_values('count-norm', ascending=False)
norm_total

,count-norm
color,
Receiver band was already locked {} {}e {},43.012698
Parking Calibration Device,41.933333
Set coarse tune {} High IFTP {} {},30.473016
Modulation voltage {} {} IFTotal Power {} {} LPR power {} {} Photomixer Current {} {},29.682540
Loop ongoing j {},28.814286
...,...
Field alma CONTROL ANTxx LORR TE_LENGTH archive_delta_percent does not exist,0.001587
Field alma CONTROL ANTxx LORR CAN_ERROR_COUNT archive_delta_percent does not exist,0.001587
Field alma CONTROL ANTxx LORR CAN_ERROR archive_delta_percent does not exist,0.001587


In [19]:
# Filtering total above THRESHOLD_MIN_AVG_COUNT
THRESHOLD_MIN_AVG_COUNT = 0.1
norm_total[ norm_total['count-norm'] >= THRESHOLD_MIN_AVG_COUNT ]

,count-norm
color,
Receiver band was already locked {} {}e {},43.012698
Parking Calibration Device,41.933333
Set coarse tune {} High IFTP {} {},30.473016
Modulation voltage {} {} IFTotal Power {} {} LPR power {} {} Photomixer Current {} {},29.682540
Loop ongoing j {},28.814286
...,...
Limiting subreflector position {} {} {} {} {} mm ensure it does not exceed limits {} {} {} mm,0.233333
Aborting data collection that started {} {} {} {} and was due stop {} {} {} {} Time now {} {} {} {},0.160317
Data acquisition aborted,0.155556


In [9]:
#colors_by_instance[["color", "count"]].groupby(["color"]).median().sort_values('count', ascending=False)[:50]